In [1]:
import cv2
import dlib
import time
import os
from PIL import Image,ImageOps
import numpy as np
import pickle

def face_data(folder):
    video=cv2.VideoCapture(0)
    
    directory = ["org","up","down","left","right"]
    parent_dir = os.path.dirname(os.path.abspath("__file__"))+"\\"+folder
  
    for dir in directory:
        path = os.path.join(parent_dir, dir)
        os.mkdir(path)
    
    a=1
    while True:
        a=a+1
        check,image=video.read()
        gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    
        hog_face_detector = dlib.get_frontal_face_detector()
        faces_hog = hog_face_detector(gray, 1)
    
        for face in faces_hog:
            x = face.left()
            y = face.top()
            w = face.right() - x
            h = face.bottom() - y
        try:
            cv2.rectangle(image, (x,y), (x+w,y+h), (0,255,0), 2)
            cv2.imshow("face detection with dlib", image)
        
            img=gray[y:y+h,x:x+w]
            he,wd=img.shape
        
            up=img[0:he//2,0:wd]
            down=img[he//2:he,0:wd]
            left=img[0:he,0:wd//2]
            right=img[0:he,wd//2:wd]
        
            cv2.imwrite(folder+"\\org\\pic"+str(a)+".jpg",img)
            cv2.imwrite(folder+"\\up\\pic"+str(a)+".jpg",up)
            cv2.imwrite(folder+"\\down\\pic"+str(a)+".jpg",down)
            cv2.imwrite(folder+"\\left\\pic"+str(a)+".jpg",left)
            cv2.imwrite(folder+"\\right\\pic"+str(a)+".jpg",right)
        except:
            pass
    
        key=cv2.waitKey(1)
        if(key==ord("q")):
            break
    
    time.sleep(3)
    video.release()
    cv2.destroyAllWindows()

In [2]:
def train(folder,model_name):
    Base_Dir=os.path.dirname(os.path.abspath("__file__"))
    image_dir=os.path.join(Base_Dir,folder)

    recognizer=cv2.face.LBPHFaceRecognizer_create()

    current_id=0
    label_ids={}
    x=[]
    y=[]

    for root,dirs,files in os.walk(image_dir):
        for file in files:
            if file.endswith("jpg"):
                path = os.path.join(root,file)
                label = os.path.basename(os.path.dirname(path)).replace(" ","-").lower()
                #print(label,path)
                if label in label_ids:
                    pass
                else:
                    label_ids[label]=current_id
                    current_id+=1
                id_=label_ids[label]
                #print(label_ids)
            
                pImage = Image.open(path)
                pGray = ImageOps.grayscale(pImage)
                pArray = np.array(pGray,"uint8")
                #print(pArray)
                x.append(pArray)
                y.append(id_)
                
    with open("labels.pickle","wb") as f:
        pickle.dump(label_ids,f)
    
    recognizer.train(x,np.array(y))
    recognizer.save(model_name+".yml")

In [3]:
def recognize(model_name):
    recognizer=cv2.face.LBPHFaceRecognizer_create()
    recognizer.read(model_name+".yml")

    video=cv2.VideoCapture(0)
    i=0
    y=0
    n=0
    while True:
        c,frame=video.read()
        
        gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        try:
            hog_face_detector = dlib.get_frontal_face_detector()
            faces_hog = hog_face_detector(gray, 1)
    
            for face in faces_hog:
                x = face.left()
                y = face.top()
                w = face.right() - x
                h = face.bottom() - y
    
                img=frame[y:y+h,x:x+w]
                gray_img=gray[y:y+h,x:x+w]
        
                id_,conf=recognizer.predict(gray_img)
        
                y+=1
                if conf>=35:
                    print(id_)
                    n+=1
        
                cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0))
        except:
            pass
        cv2.imshow("Frame",frame)
        key=cv2.waitKey(20)
        if key == ord('q'):
            break

    print(y,"-",n)
    video.release()
    cv2.destroyAllWindows()

In [4]:
face_data("data")

In [5]:
train("data","model")

In [6]:
recognize("model")

2
2
2
2
2
2
223 - 6
